In [1]:
# %pip install cloudscraper

import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import re
import cloudscraper
import time
import random
import csv

SEASON_LIST = ['2016', '2017', '2018', '2019', '2020'] # can be tweaked still

WEEK_LIST = ['REG1', 'REG2', 'REG3', 'REG4', 'REG5', 'REG6', 'REG7', 'REG8', 'REG9', 'REG10',
             'REG11', 'REG12', 'REG13', 'REG14', 'REG15', 'REG16', 'REG17', 
             'POST1', 'POST2', 'POST3', 'POST4', 'POST5']

SEASON_URL = 'https://www.nfl.com/injuries/league/{}/{}'


In [2]:
# Create a scraper that bypasses Cloudflare
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'windows',
        'mobile': False
    }
)

year_add = ["january", "february"]

for season in SEASON_LIST:
    for week in WEEK_LIST:
        url = SEASON_URL.format(season, week)
        # print(url)

        time.sleep(random.uniform(2, 4))

        try:
            response = scraper.get(url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                print(f"Successfully fetched {url}")

                week_injury_div = soup.find('div', {'class': 'nfl-o-injury-report__wrap'})
                if week_injury_div:
                    # print(week_injury_div)
                    # print(f"Found injuries for {season}, {week}")

                    games = week_injury_div.find_all('section', {'class': 'nfl-o-injury-report__unit'})
                    # print(len(games))

                    for game in games:

                        closest_date = game.find_previous('h2', {'class': 'd3-o-section-title'}).text.strip()
                        month = closest_date.split(' ')[-2]

                        season_int = int(season)
                        # print(type(season))
                        if month.lower() in year_add:
                            season_int += 1

                        closest_date_edited = closest_date.lower().replace(',', '')[:-2] + ', ' + str(season_int)

                        team_tables = game.find_all('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-reports--detailed'})
                        for table in team_tables:

                            closest_team_title = table.find_previous('div', {'class': 'd3-o-section-sub-title'}).text.strip()
                            # print(closest_date)
                            tbody = table.find('tbody')
                            rows = tbody.find_all('tr')

                            for row in rows:
                                td_elements = row.find_all('td')

                                player_name = td_elements[0].get_text(strip=True)
                                position = td_elements[1].get_text(strip=True)
                                injuries = td_elements[2].get_text(strip=True)
                                practice_status = td_elements[3].get_text(strip=True)
                                game_status = td_elements[4].get_text(strip=True)

                                # break

                                with open('player_injury_data.csv', 'a', newline='', encoding='utf-8') as f:
                                    writer = csv.writer(f)

                                    writer.writerow([
                                        player_name,
                                        closest_date_edited,
                                        closest_team_title,
                                        position,
                                        injuries,
                                        practice_status,
                                        game_status
                                    ])

                            # print(table, '\n\n\n\n\n\n\n\n')

                        # break

            else:
                print(f"Failed with status: {response.status_code}")
        except Exception as e:
            print(f"Error: {e}")

        # break
    # break


Successfully fetched https://www.nfl.com/injuries/league/2016/REG1
Successfully fetched https://www.nfl.com/injuries/league/2016/REG2
Successfully fetched https://www.nfl.com/injuries/league/2016/REG3
Successfully fetched https://www.nfl.com/injuries/league/2016/REG4
Successfully fetched https://www.nfl.com/injuries/league/2016/REG5
Successfully fetched https://www.nfl.com/injuries/league/2016/REG6
Successfully fetched https://www.nfl.com/injuries/league/2016/REG7
Successfully fetched https://www.nfl.com/injuries/league/2016/REG8
Successfully fetched https://www.nfl.com/injuries/league/2016/REG9
Successfully fetched https://www.nfl.com/injuries/league/2016/REG10
Successfully fetched https://www.nfl.com/injuries/league/2016/REG11
Successfully fetched https://www.nfl.com/injuries/league/2016/REG12
Successfully fetched https://www.nfl.com/injuries/league/2016/REG13
Successfully fetched https://www.nfl.com/injuries/league/2016/REG14
Successfully fetched https://www.nfl.com/injuries/league/